***PART 1. NEURAL NETWORKS***

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        
        # Define various layers here
        self.fc1 = nn.Linear(28*28, 19) #\
        self.fc2 = nn.Linear(19, 19)    #  > 19 Output Size to meet the Less than 20 Neurons Requirement
        self.fc3 = nn.Linear(19, 10)    #/

        self.dp1 = nn.Linear(28*28, 201)  #\
        self.dp2 = nn.Linear(201, 201)    # \ 201 Output Size to meet the More than 200 Neurons Requirement
        self.dp3 = nn.Linear(201, 201)    # /
        self.dp4 = nn.Linear(201, 10)     #/
        
        # This will select the forward pass function based on mode for the ConvNet.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
      
    # Baseline sample model
    def model_0(self, X):
        # ======================================================================
        # Three fully connected layers with activation
        
        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = torch.sigmoid(X)
                
        return X  
        
    # Baseline model. task 1
    def model_1(self, X):
        # ======================================================================
        # Three fully connected layers without activation

        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)
        X = self.fc2(X)
        X = self.fc3(X)
        # NotImplementedError        
                        
        return X
        

    # task 2
    def model_2(self, X):
        # ======================================================================
        # Train with activation (use model 1 from task 1)
        
        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)
        X = F.relu(X)
        X = self.fc2(X)
        X = F.relu(X)
        X = self.fc3(X)
        X = F.relu(X)
        # NotImplementedError
        
        return X

	
    # task 3
    def model_3(self, X):
        # ======================================================================
        # Change number of fully connected layers and number of neurons from model 2 in task 2
        
        X = torch.flatten(X, start_dim=1)
        X = self.dp1(X)
        X = F.relu(X)
        X = self.dp2(X)
        X = F.relu(X)
        X = self.dp3(X)
        X = F.relu(X)
        X = self.dp4(X)
        X = F.relu(X)
        # NotImplementedError
        
        return X

In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet 
import argparse
import numpy as np 


def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        
        # ======================================================================
        # Compute loss based on criterion
        # ----------------- YOUR CODE HERE ----------------------
        #
        # Remove NotImplementedError and assign correct loss function.
        loss = criterion(output, target, reduction='mean')
        
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # ======================================================================
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc
    


def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            

            # Predict for data by doing forward pass
            output = model(data)
            
            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # ======================================================================
            # Count correct predictions overall 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy
    

def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    
    # Set proper device based on cuda availability 
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)
    
    # Initialize the model and send to device 
    model = ConvNet(FLAGS.mode).to(device)
    
    # Initialize the criterion for loss computation 
    # ======================================================================
    # Remove NotImplementedError and assign correct loss function.
    criterion = F.cross_entropy
    
    
    # Initialize optimizer type 
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)
    
    # Create transformations to apply to each data sample 
    # Can specify variations such as image flip, color flip, random crop, ...
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    
    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.MNIST('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size, 
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size, 
                                shuffle=False, num_workers=4)
    
    best_accuracy = 0.0
    
    # Run training for n_epochs specified in config 
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            
    print("accuracy is {:2.2f}".format(best_accuracy))
    
    print("Training and evaluation finished")
    
    
if __name__ == '__main__':
    # Set parameters for Sparse Autoencoder
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--mode',
                        type=int, default=3,
                        help='Select mode between 1-3.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.1,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=20,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=10,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str,
                        default='logs',
                        help='Directory to put logging.')
                        
    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()
    
    print("Mode: ", FLAGS.mode)
    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)
    
    run_main(FLAGS)

Mode:  3
LR:  0.1
Batch size:  10
Torch device selected:  cuda

Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train set: Average loss: 0.2907, Accuracy: 54676/60000 (91%)
Test set: Average loss: 0.1288, Accuracy: 9607/10000 (96%)

Epoch:  2
Train set: Average loss: 0.1146, Accuracy: 57995/60000 (97%)
Test set: Average loss: 0.1026, Accuracy: 9697/10000 (97%)

Epoch:  3
Train set: Average loss: 0.0789, Accuracy: 58630/60000 (98%)
Test set: Average loss: 0.1642, Accuracy: 9611/10000 (96%)

Epoch:  4
Train set: Average loss: 0.0671, Accuracy: 58788/60000 (98%)
Test set: Average loss: 0.2022, Accuracy: 9560/10000 (96%)

Epoch:  5
Train set: Average loss: 0.0516, Accuracy: 59090/60000 (98%)
Test set: Average loss: 0.1159, Accuracy: 9722/10000 (97%)

Epoch:  6
Train set: Average loss: 0.0485, Accuracy: 59184/60000 (99%)
Test set: Average loss: 0.1030, Accuracy: 9726/10000 (97%)

Epoch:  7
Train set: Average loss: 0.0366, Accuracy: 59335/60000 (99%)
Test set: Average loss: 0.0958, Accuracy: 9772/10000 (98%)

Epoch:  8
Train set: Average loss: 0.0335, Accuracy: 59386/60000 (99%)
Test set: Average loss

***Part 1 Report***

***Model 1***

13 Min 50 Sec for Completion

Final Epoch Accuracy:

Train: 10%

Test: 10%

Best Accuracy: 9.8%

Note: Model Converged after Epoch 2 (No change in Epochs 3+)

Both Losses were nan for all epochs

--------------------------------------------------------

***Model 2***

14 Min 33 Sec for Completion

Final Epoch Accuracy:

Train: 95%

Test: 95%

Best Accuracy: 95.26%

Both Losses stayed consistent around 0.2 - 0.3 all throughout after Epoch 1

--------------------------------------------------------

***Model 3***

15 Min 22 Sec for Completion

Final Epoch Accuracy:

Train: 100%

Test: 97%

Best Accuracy: 97.88%

Training Loss decreased to the 0.01 - 0.02 range

Testing Loss was in the 0.1 - 0.2 range


***Part 1 Discussion***

Model 3 was the BEST MODEL

Increasing the complexity of the model increased the runtime, but only by about a minute for each upgrade.

Adding the activation function resulted in a dramatic increase in accuracy, and making the model deeper provided that last bit to get to almost 100% accuracy.

Better Model = Lower Loss = Higher Accuracy

***PART 2. CONVOLUTIONAL NEURAL NETWORK***

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        
        # Define various layers here, such as in the tutorial example
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        #self.conv2 = Make conv layer like above
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)
        #self.conv3 = Make conv layer like above
        self.conv1Big = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3)
        self.conv2Big = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)
        self.conv3Big = nn.Conv2d(in_channels=40, out_channels=40, kernel_size=3)
        
        self.fc1_model1 = nn.Linear(360, 100)  # This is first fully connected layer for step 1.
        self.fc1_model2 = nn.Linear(1440, 100) # This is first fully connected layer for step 2.
        self.fc1_model3 = nn.Linear(640, 100)  # This is first fully connected layer for step 3
        
        self.fc2 = nn.Linear(100, 10)       # This is 2nd fully connected layer for all models.
        
        self.fc_model0 = nn.Linear(2250, 100)   # This is for example model.
        
        
        # This will select the forward pass function based on mode for the ConvNet.
        # Based on the question, you have 3 modes available for step 1 to 3.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        elif mode == 0:
            self.forward = self.model_0
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
        
    
    # Example model. Modify this for step 1-3
    def model_0(self, X):
        # ======================================================================         
        
        X = F.relu(self.conv1(X))
        #print(X.shape)
        X = F.max_pool2d(X, kernel_size=2)
        #print(X.shape)
        
        X = torch.flatten(X, start_dim=1)
        #print(X.shape)
        
        X = F.relu(self.fc_model0(X))
        X = self.fc2(X)
        
        return X
        
    
    # Simple CNN. step 1
    def model_1(self, X):
        # ======================================================================
         
        # Complete this part as model_0, add one more conv2d layer 
        # with relu activation followed by maxpool layer.
        
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, kernel_size=2)
        
        X = torch.flatten(X, start_dim=1)
        
        X = F.relu(self.fc1_model1(X))
        X = self.fc2(X)
        
        return X
        

    # Increase filters. step 2
    def model_2(self, X):
        # ======================================================================
        
        # Complete this part as model_1. Modify in/out channels for conv2d layers.
        
        X = F.relu(self.conv1Big(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv2Big(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = torch.flatten(X, start_dim=1)

        X = F.relu(self.fc1_model2(X))
        X = self.fc2(X)
        
        return X
        

    # Large CNN. step 3
    def model_3(self, X):
        # ======================================================================
        
        # Complete this part as model_2, add one more conv2d layer 
        # with relu activation. Do not add maxpool after this new conv2d layer.
        
        X = F.relu(self.conv1Big(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv2Big(X))
        X = F.max_pool2d(X, kernel_size=2)

        X = F.relu(self.conv3Big(X))

        X = torch.flatten(X, start_dim=1)

        X = F.relu(self.fc1_model3(X))
        X = self.fc2(X)
        
        return X

In [ ]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter
#from ConvNet import ConvNet 
import argparse
import numpy as np 

def train(model, device, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        data, target = batch_sample
        
        # Push data/label to correct device
        data, target = data.to(device), target.to(device)
        # print(data.shape)
        # print(target.shape)
        # exit()
        
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)
        
        # ======================================================================
        # Compute loss based on criterion
        loss = criterion(output, target)
        
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # ======================================================================
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    return train_loss, train_acc
    


def test(model, device, test_loader):
    '''
    Tests the model.
    model: The model to train. Should already be in correct device.
    device: 'cuda' or 'cpu'.
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
        for batch_idx, sample in enumerate(test_loader):
            data, target = sample
            data, target = data.to(device), target.to(device)
            

            # Predict for data by doing forward pass
            output = model(data)
            
            # ======================================================================
            # Compute loss based on same criterion as training
            loss = F.cross_entropy(output, target, reduction='mean')
            
            # Append loss to overall test loss
            losses.append(loss.item())
            
            # Get predicted index by selecting maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # ======================================================================
            # Count correct predictions overall 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy
    

def run_main(FLAGS):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    
    # Set proper device based on cuda availability 
    device = torch.device("cuda" if use_cuda else "cpu")
    print("Torch device selected: ", device)
    
    # Initialize the model and send to device 
    model = ConvNet(FLAGS.mode).to(device)
    # print(model)
    # exit()

    # Initialize the criterion for loss computation 
    criterion = nn.CrossEntropyLoss(reduction='mean')
    
    # Initialize optimizer type 
    optimizer = optim.SGD(model.parameters(), lr=FLAGS.learning_rate, weight_decay=1e-7)
    
    # Create transformations to apply to each data sample 
    # Can specify variations such as image flip, color flip, random crop, ...
    #transform=transforms.Compose([
    #    transforms.ToTensor(),
    #    transforms.Normalize((0.1307,), (0.3081,))
    #    ])
    
    transform = transforms.Compose(
                    [transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
     
    # Load datasets for training and testing
    # Inbuilt datasets available in torchvision (check documentation online)
    dataset1 = datasets.CIFAR10('./data/', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.CIFAR10('./data/', train=False,
                       transform=transform)
    train_loader = DataLoader(dataset1, batch_size = FLAGS.batch_size, 
                                shuffle=True, num_workers=4)
    test_loader = DataLoader(dataset2, batch_size = FLAGS.batch_size, 
                                shuffle=False, num_workers=4)
    
    best_accuracy = 0.0
    
    # Run training for n_epochs specified in config 
    for epoch in range(1, FLAGS.num_epochs + 1):
        print("\nEpoch: ", epoch)
        train_loss, train_accuracy = train(model, device, train_loader,
                                            optimizer, criterion, epoch, FLAGS.batch_size)
        test_loss, test_accuracy = test(model, device, test_loader)
        
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            
    print("accuracy is {:2.2f}".format(best_accuracy))
    
    print("Training and evaluation finished")
    
    
if __name__ == '__main__':
    # Set parameters for Sparse Autoencoder
    parser = argparse.ArgumentParser('CNN Exercise.')
    parser.add_argument('--mode',
                        type=int, default=3,
                        help='Select mode between 1-3.')
    parser.add_argument('--learning_rate',
                        type=float, default=0.1,
                        help='Initial learning rate.')
    parser.add_argument('--num_epochs',
                        type=int,
                        default=10,
                        help='Number of epochs to run trainer.')
    parser.add_argument('--batch_size',
                        type=int, default=100,
                        help='Batch size. Must divide evenly into the dataset sizes.')
    parser.add_argument('--log_dir',
                        type=str,
                        default='logs',
                        help='Directory to put logging.')
                        
    FLAGS = None
    FLAGS, unparsed = parser.parse_known_args()
    
    print("Mode: ", FLAGS.mode)
    print("LR: ", FLAGS.learning_rate)
    print("Batch size: ", FLAGS.batch_size)
    
    run_main(FLAGS)

Mode:  3
LR:  0.1
Batch size:  100
Torch device selected:  cuda
Files already downloaded and verified

Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train set: Average loss: 1.9687, Accuracy: 13989/50000 (28%)
Test set: Average loss: 1.5633, Accuracy: 4312/10000 (43%)

Epoch:  2
Train set: Average loss: 1.4880, Accuracy: 23100/50000 (46%)
Test set: Average loss: 1.3529, Accuracy: 5121/10000 (51%)

Epoch:  3
Train set: Average loss: 1.2971, Accuracy: 26822/50000 (54%)
Test set: Average loss: 1.2618, Accuracy: 5478/10000 (55%)

Epoch:  4
Train set: Average loss: 1.1653, Accuracy: 29482/50000 (59%)
Test set: Average loss: 1.0930, Accuracy: 6129/10000 (61%)

Epoch:  5
Train set: Average loss: 1.0650, Accuracy: 31284/50000 (63%)
Test set: Average loss: 1.0284, Accuracy: 6353/10000 (64%)

Epoch:  6
Train set: Average loss: 0.9811, Accuracy: 32718/50000 (65%)
Test set: Average loss: 0.9929, Accuracy: 6472/10000 (65%)

Epoch:  7
Train set: Average loss: 0.9122, Accuracy: 34004/50000 (68%)
Test set: Average loss: 0.9523, Accuracy: 6676/10000 (67%)

Epoch:  8
Train set: Average loss: 0.8474, Accuracy: 35212/50000 (70%)
Test set: Average loss

***Part 2 Report***

***Model 1***

2 Min 36 Sec for Completion

Final Epoch Accuracy:

Train: 69%

Test: 65%

Best Accuracy: 64.61%

Training Loss decreased consistently to 0.8897

Testing Loss was in the 1.0 - 1.1 range

--------------------------------------------------------

***Model 2***

2 Min 38 Sec for Completion

Final Epoch Accuracy:

Train: 79%

Test: 68%

Best Accuracy: 68.72%

Training Loss decreased consistently to 0.5975

Testing Loss was in the 0.9 - 1.0 range

--------------------------------------------------------

***Model 3***

2 Min 42 Sec for Completion

Final Epoch Accuracy:

Train: 74%

Test: 68%

Best Accuracy: 68.88%

Training Loss decreased consistently to 0.7434

Testing Loss was in the 0.8 - 1.0 range


***Part 2 Discussion***

The BEST MODEL is unclear. Model 2 performed by far the best on the training data, but was approximately equal to Model 3 when it came to the testing data.

Increasing the complexity of the model did not increase runtime. The runtime of this CNN was significantly faster than the NN of PART 1.

Increasing the features of the layers provided a significant boost in accuracy while increasing the number of layers showed no statistical significance (at least when the additional layer has the same input and output features).